# 文本相似度实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [2]:
dataset = load_dataset("json", data_files="./train_pair_1w.json", split="train")
dataset

Found cached dataset json (C:/Users/yuyao/.cache/huggingface/datasets/json/default-6c4b70959665864d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 10000
})

In [3]:
dataset[0]

{'sentence1': '找一部小时候的动画片', 'sentence2': '求一部小时候的动画片。谢了', 'label': '1'}

## Step3 划分数据集

In [4]:
datasets = dataset.train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 2000
    })
})

## Step4 数据集预处理

In [5]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

def process_function(examples):
    sentences = []
    labels = []
    for sen1, sen2, label in zip(examples["sentence1"], examples["sentence2"], examples["label"]):
        sentences.append(sen1)
        sentences.append(sen2)
        labels.append(1 if int(label) == 1 else -1)
    # input_ids, attention_mask, token_type_ids
    tokenized_examples = tokenizer(sentences, max_length=128, truncation=True, padding="max_length")
    tokenized_examples = {k: [v[i: i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}
    tokenized_examples["labels"] = labels
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [6]:
print(tokenized_datasets["train"][0])

{'input_ids': [[101, 3221, 671, 702, 1962, 1765, 3175, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2769, 671, 4684, 5287, 5383, 8024, 800, 812, 3221, 2582, 3416, 2418, 802, 6814, 3341, 4638, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Step5 创建模型

In [14]:
from transformers import BertForSequenceClassification, BertPreTrainedModel, BertModel
from typing import Optional
from transformers.configuration_utils import PretrainedConfig
from torch.nn import CosineSimilarity, CosineEmbeddingLoss

class DualModel(BertPreTrainedModel):

    def __init__(self, config: PretrainedConfig, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = BertModel(config)
        self.post_init()

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # Step1 分别获取sentenceA 和 sentenceB的输入
        senA_input_ids, senB_input_ids = input_ids[:, 0], input_ids[:, 1]
        senA_attention_mask, senB_attention_mask = attention_mask[:, 0], attention_mask[:, 1]
        senA_token_type_ids, senB_token_type_ids = token_type_ids[:, 0], token_type_ids[:, 1]

        # Step2 分别获取sentenceA 和 sentenceB的向量表示
        senA_outputs = self.bert(
            senA_input_ids,
            attention_mask=senA_attention_mask,
            token_type_ids=senA_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        senA_pooled_output = senA_outputs[1]    # [batch, hidden]

        senB_outputs = self.bert(
            senB_input_ids,
            attention_mask=senB_attention_mask,
            token_type_ids=senB_token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        senB_pooled_output = senB_outputs[1]    # [batch, hidden]

        # step3 计算相似度

        cos = CosineSimilarity()(senA_pooled_output, senB_pooled_output)    # [batch, ]

        # step4 计算loss

        loss = None
        if labels is not None:
            loss_fct = CosineEmbeddingLoss(0.3)
            loss = loss_fct(senA_pooled_output, senB_pooled_output, labels)

        output = (cos,)
        return ((loss,) + output) if loss is not None else output
    
model = DualModel.from_pretrained("hfl/chinese-macbert-base")

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing DualModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing DualModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DualModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Step6 创建评估函数

In [8]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

In [13]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = [int(p > 0.7) for p in predictions]
    labels = [int(l > 0) for l in labels]
    # predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [10]:
train_args = TrainingArguments(output_dir="./dual_model",      # 输出文件夹
                               per_device_train_batch_size=32,  # 训练时的batch_size
                               per_device_eval_batch_size=32,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型

## Step8 创建Trainer

In [16]:
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  compute_metrics=eval_metric)

## Step9 模型训练

In [17]:
trainer.train()

c:\Users\yuyao\miniconda3\envs\transformers\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/750 [00:00<?, ?it/s]

{'loss': 0.3393, 'learning_rate': 1.9733333333333336e-05, 'epoch': 0.04}
{'loss': 0.3197, 'learning_rate': 1.9466666666666668e-05, 'epoch': 0.08}
{'loss': 0.2971, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.12}
{'loss': 0.2934, 'learning_rate': 1.8933333333333334e-05, 'epoch': 0.16}
{'loss': 0.2855, 'learning_rate': 1.866666666666667e-05, 'epoch': 0.2}
{'loss': 0.2488, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.24}
{'loss': 0.2524, 'learning_rate': 1.8133333333333335e-05, 'epoch': 0.28}
{'loss': 0.2468, 'learning_rate': 1.7866666666666666e-05, 'epoch': 0.32}
{'loss': 0.2605, 'learning_rate': 1.76e-05, 'epoch': 0.36}
{'loss': 0.2281, 'learning_rate': 1.7333333333333336e-05, 'epoch': 0.4}
{'loss': 0.2411, 'learning_rate': 1.706666666666667e-05, 'epoch': 0.44}
{'loss': 0.2495, 'learning_rate': 1.6800000000000002e-05, 'epoch': 0.48}
{'loss': 0.2328, 'learning_rate': 1.6533333333333333e-05, 'epoch': 0.52}
{'loss': 0.2097, 'learning_rate': 1.6266666666666668e-05, 'epoch': 0.

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.17725127935409546, 'eval_accuracy': 0.8045, 'eval_f1': 0.759975445058318, 'eval_runtime': 3.3068, 'eval_samples_per_second': 604.82, 'eval_steps_per_second': 19.052, 'epoch': 1.0}
{'loss': 0.142, 'learning_rate': 1.3066666666666668e-05, 'epoch': 1.04}
{'loss': 0.1731, 'learning_rate': 1.2800000000000001e-05, 'epoch': 1.08}
{'loss': 0.1782, 'learning_rate': 1.2533333333333336e-05, 'epoch': 1.12}
{'loss': 0.1738, 'learning_rate': 1.2266666666666667e-05, 'epoch': 1.16}
{'loss': 0.1647, 'learning_rate': 1.2e-05, 'epoch': 1.2}
{'loss': 0.1478, 'learning_rate': 1.1733333333333335e-05, 'epoch': 1.24}
{'loss': 0.1662, 'learning_rate': 1.1466666666666668e-05, 'epoch': 1.28}
{'loss': 0.1613, 'learning_rate': 1.1200000000000001e-05, 'epoch': 1.32}
{'loss': 0.1642, 'learning_rate': 1.0933333333333334e-05, 'epoch': 1.36}
{'loss': 0.1589, 'learning_rate': 1.0666666666666667e-05, 'epoch': 1.4}
{'loss': 0.1681, 'learning_rate': 1.04e-05, 'epoch': 1.44}
{'loss': 0.1673, 'learning_rate':

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.16970285773277283, 'eval_accuracy': 0.808, 'eval_f1': 0.7675544794188862, 'eval_runtime': 3.2879, 'eval_samples_per_second': 608.284, 'eval_steps_per_second': 19.161, 'epoch': 2.0}
{'loss': 0.1267, 'learning_rate': 6.4000000000000006e-06, 'epoch': 2.04}
{'loss': 0.1509, 'learning_rate': 6.133333333333334e-06, 'epoch': 2.08}
{'loss': 0.1275, 'learning_rate': 5.8666666666666675e-06, 'epoch': 2.12}
{'loss': 0.1353, 'learning_rate': 5.600000000000001e-06, 'epoch': 2.16}
{'loss': 0.1224, 'learning_rate': 5.333333333333334e-06, 'epoch': 2.2}
{'loss': 0.1224, 'learning_rate': 5.0666666666666676e-06, 'epoch': 2.24}
{'loss': 0.1209, 'learning_rate': 4.800000000000001e-06, 'epoch': 2.28}
{'loss': 0.1288, 'learning_rate': 4.533333333333334e-06, 'epoch': 2.32}
{'loss': 0.116, 'learning_rate': 4.266666666666668e-06, 'epoch': 2.36}
{'loss': 0.1475, 'learning_rate': 4.000000000000001e-06, 'epoch': 2.4}
{'loss': 0.1229, 'learning_rate': 3.7333333333333337e-06, 'epoch': 2.44}
{'loss': 0

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.16770052909851074, 'eval_accuracy': 0.813, 'eval_f1': 0.7688504326328801, 'eval_runtime': 3.285, 'eval_samples_per_second': 608.836, 'eval_steps_per_second': 19.178, 'epoch': 3.0}
{'train_runtime': 156.16, 'train_samples_per_second': 153.689, 'train_steps_per_second': 4.803, 'train_loss': 0.1770754319826762, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.1770754319826762, metrics={'train_runtime': 156.16, 'train_samples_per_second': 153.689, 'train_steps_per_second': 4.803, 'train_loss': 0.1770754319826762, 'epoch': 3.0})

## Step10 模型评估

In [18]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.16770052909851074,
 'eval_accuracy': 0.813,
 'eval_f1': 0.7688504326328801,
 'eval_runtime': 3.3978,
 'eval_samples_per_second': 588.616,
 'eval_steps_per_second': 18.541,
 'epoch': 3.0}

## Step11 模型预测

In [30]:
class SentenceSimilarityPipeline:

    def __init__(self, model, tokenizer) -> None:
        self.model = model.bert
        self.tokenizer = tokenizer
        self.device = model.device

    def preprocess(self, senA, senB):
        return self.tokenizer([senA, senB], max_length=128, truncation=True, return_tensors="pt", padding=True)

    def predict(self, inputs):
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        return self.model(**inputs)[1]  # [2, 768]

    def postprocess(self, logits):
        cos = CosineSimilarity()(logits[None, 0, :], logits[None,1, :]).squeeze().cpu().item()
        return cos

    def __call__(self, senA, senB, return_vector=False):
        inputs = self.preprocess(senA, senB)
        logits = self.predict(inputs)
        result = self.postprocess(logits)
        if return_vector:
            return result, logits
        else:
            return result

In [31]:
pipe = SentenceSimilarityPipeline(model, tokenizer)

In [35]:
pipe("我喜欢北京", "明天不行", return_vector=True)

(-0.03769354894757271,
 tensor([[ 0.3857, -0.6741,  0.9823,  ..., -0.5434, -0.8953, -0.2401],
         [-0.9959, -0.9424, -0.6970,  ...,  0.9715,  0.9695, -0.8788]],
        device='cuda:0', grad_fn=<TanhBackward0>))